In [ ]:
import pandas as pd
# install needed libraries into your venv (run only once or when missing)
!pip install sentence-transformers faiss-cpu numpy python-dotenv openai

import os, glob, pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from dotenv import load_dotenv
from openai import OpenAI

print("Libraries imported.")


In [ ]:
import pandas as pd

DATA_PATH = "Data/galgotias_faq_3000.csv"
faq_df = pd.read_csv(DATA_PATH)

print("Rows:", len(faq_df))
faq_df.head()


In [ ]:
docs = []
metadata = []

for _, row in faq_df.iterrows():
    q = row["question"]
    a = row["answer"]
    text = f"Q: {q}\nA: {a}"
    docs.append(text)
    metadata.append({"id": row["id"], "category": row["category"]})

print("Docs loaded:", len(docs))


In [ ]:
print("Loading Sentence-BERT model...")
embed_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

print("Encoding documents...")
embeddings = embed_model.encode(docs, show_progress_bar=True, convert_to_numpy=True)

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("Index size (vectors):", index.ntotal)


In [ ]:
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

K = 3  # top-k docs

def retrieve_context(query, k=K):
    q_emb = embed_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(q_emb, k)
    return [docs[i] for i in indices[0]]

def generate_answer(query, k=K):
    context = "\n\n".join(retrieve_context(query, k))

    prompt = f"""
You are an enquiry bot for Galgotias University.
Use ONLY the information in the context to answer the question.
If the answer is not in the context, clearly say you don't know.

CONTEXT:
{context}

QUESTION:
{query}

ANSWER:
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",   # ✅ valid, supported Groq model
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

def ask(q):
    print("Q:", q)
    print("\nA:", generate_answer(q))


In [ ]:
ask("What is the admission process for B.Tech?")
ask("Is hostel facility available for students?")


In [ ]:
ask("What is the admission process for B.Tech?")
ask("Is hostel facility available for students?")
ask("What is the annual fee for B.Tech CSE?")
ask("Does the university provide transport facilities?")


In [ ]:
while True:
    q = input("\nYou: ")
    if q.lower() in ["exit", "quit", "bye"]:
        print("Bot: Bye 👋")
        break
    print("Bot:", generate_answer(q))
